Construccion de modelos de machine learning para el dataset "kc house data"

In [187]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import locale
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [188]:
# read the data
df = pd.read_csv("kc_house_data.csv")
df.shape

(21613, 21)

El dataset consta de 21 columnas o características y abarca 21,613 registros en total. Cada registro representa una entrada única en el conjunto de datos, y nuestro análisis exploratorio y modelos de machine learning se enfocarán en extraer información valiosa de esta estructura de datos.

In [189]:
numeric_variables = df.select_dtypes(include=['float64', 'int64']).columns
categorical_variables = df.select_dtypes(include=['object']).columns

num_numeric_variables = len(numeric_variables)
num_categorical_variables = len(categorical_variables)

print("Número de variables numéricas:", num_numeric_variables)
print("Número de variables categóricas:", num_categorical_variables)

numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

print("Columnas numéricas:", numeric_columns)
print("Columnas categóricas:", categorical_columns)

Número de variables numéricas: 20
Número de variables categóricas: 1
Columnas numéricas: ['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']
Columnas categóricas: ['date']


In [190]:
# Verificar campos vacíos por columna
empty_columns = df.columns[df.isnull().any()]
empty_counts = df[empty_columns].isnull().sum()

# Verificar valores nulos en total
total_missing_values = df.isnull().sum().sum()

print("Campos vacíos por columna:")
print(empty_counts)
print("\nTotal de valores nulos:", total_missing_values)

Campos vacíos por columna:
Series([], dtype: float64)

Total de valores nulos: 0


In [191]:
# Obtener la cantidad de valores únicos en cada columna y ordenar de menor a mayor
unique_value_counts_sorted = df.nunique().sort_values()

# Imprimir los resultados ordenados
print(unique_value_counts_sorted)
# 

waterfront           2
view                 5
condition            5
floors               6
grade               12
bedrooms            13
bathrooms           30
zipcode             70
yr_renovated        70
yr_built           116
sqft_basement      306
date               372
long               752
sqft_living15      777
sqft_above         946
sqft_living       1038
price             4028
lat               5034
sqft_lot15        8689
sqft_lot          9782
id               21436
dtype: int64


La mayoría de las casas tienen 3 o 4 dormitorios, estas son configuraciones comunes en las viviendas.

Las casas con 1 o 2 dormitorios también son relativamente comunes, lo que podría indicar la presencia de apartamentos o propiedades más pequeñas en el conjunto de datos.

La cantidad de casas con 5 o más dormitorios disminuye a medida que el número de dormitorios aumenta. 

Esto podría reflejar una disminución en la demanda o disponibilidad de casas con más dormitorios.

La presencia de casas con 0 dormitorios podría ser un error en los datos o podría indicar propiedades no residenciales, como terrenos vacantes.

La presencia de una casa con 33 dormitorios podría ser un error de entrada o un valor atípico.

In [193]:

# analisis de la variable 'waterfront' (vista al agua)


# Calcular la cantidad de casas con vista al agua y sin vista al agua
waterfront_counts = df['waterfront'].value_counts()

# Crear una lista de tuplas con los datos
data = [("Con Vista", waterfront_counts[1]), ("Sin Vista", waterfront_counts[0])]

# Crear la tabla en formato Markdown
table = tabulate(data, headers=["Tipo de Vista", "Cantidad de Casas"], tablefmt="github")

# Imprimir la tabla
print(table)


| Tipo de Vista   |   Cantidad de Casas |
|-----------------|---------------------|
| Con Vista       |                 163 |
| Sin Vista       |               21450 |


In [194]:

# Establecer el locale para el formato de moneda
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Calcular el precio promedio por tipo de vista al agua
average_price_by_waterfront = df.groupby('waterfront')['price'].mean()

# Crear una lista de tuplas con los datos
data = [("Sin Vista", locale.currency(average_price_by_waterfront[0], grouping=True)),
        ("Con Vista", locale.currency(average_price_by_waterfront[1], grouping=True))]

# Crear la tabla en formato Markdown
table = tabulate(data, headers=["Tipo de Vista", "Precio Promedio"], tablefmt="github")

# Imprimir la tabla
print(table)



| Tipo de Vista   | Precio Promedio   |
|-----------------|-------------------|
| Sin Vista       | $531,563.60       |
| Con Vista       | $1,661,876.02     |


### Analisis de la variable "waterfront"

esta variable indica si la vivienda tiene vista al agua o no, 

podemos observar que es una variable categorica
se puede observar que la mayoria de viviendas no tienen vista al agua

## Estructuramos los datos


| Columna        | Descripción                                                                                   |
|----------------|-----------------------------------------------------------------------------------------------|
| id             | ID único para cada casa vendida                                                              |
| date           | Fecha de la venta de la casa                                                                  |
| price          | Precio de cada casa vendida                                                                   |
| bedrooms       | Número de dormitorios                                                                         |
| bathrooms      | Número de baños, donde .5 representa un cuarto con inodoro pero sin ducha                   |
| sqft_living    | Metraje cuadrado del espacio interior del apartamento                                         |
| sqft_lot       | Metraje cuadrado del terreno                                                                 |
| floors         | Número de pisos                                                                               |
| waterfront     | Variable que indica si el apartamento tiene vista al agua o no                     |
| view           | Un índice del 0 al 4 que indica la calidad de la vista de la propiedad. 0 = Sin vista, 1 = Regular, 2 = Promedio, 3 = Buena, 4 = Excelente    |
| condition      | Un índice del 1 al 5 sobre la condición del apartamento. 1 = Mal estado, 2 = Regular - Mal estado, 3 = Promedio, 4 = Buen estado, 5 = Muy buen estado |
| grade          | Un índice del 1 al 13, donde 1-3 indica construcción y diseño insuficiente, 7 es promedio y 11-13 es alta calidad de construcción y diseño |
| sqft_above     | Metraje cuadrado del espacio interior sobre el nivel del suelo                               |
| sqft_basement  | Metraje cuadrado del espacio interior bajo el nivel del suelo                                 |
| yr_built       | Año en que se construyó la casa                                                               |
| yr_renovated   | Año de la última renovación de la casa                                                        |
| zipcode        | Área del código postal en la que se encuentra la casa                                         |


In [195]:
#calculamos la edad de la casa al momento de ser vendida
df["date"] = pd.to_datetime(df["date"])
df["house_age"] = df["date"].dt.year - df["yr_built"]


In [196]:
#convertimos la columna grade a categorica
def map_grade_to_category(grade):
    if grade in range(1, 4):
        return 'Construcción y Diseño Insuficiente'
    elif grade == 7:
        return 'Promedio'
    elif grade in range(11, 14):
        return 'Alta Calidad de Construcción y Diseño'
    else:
        return 'Otro'

df['grade_category'] = df['grade'].apply(map_grade_to_category)

In [ ]:
# detallando la variable bedrooms
unique_bedrooms = df['bedrooms'].unique()
print(unique_bedrooms)
bedrooms_counts = df['bedrooms'].value_counts()
print(bedrooms_counts)

[ 3  2  4  5  1  6  7  0  8  9 11 10 33]
bedrooms
3     9824
4     6882
2     2760
5     1601
6      272
1      199
7       38
0       13
8       13
9        6
10       3
11       1
33       1
Name: count, dtype: int64


In [199]:
#eliminamos los 0 dormitorios porque no serian una casa
df= df[df["bedrooms"]!= 0]
df.reset_index(inplace=True)

In [197]:
# Categorizar la variable "bedrooms"
bins = [0, 2, 4, 6, float('inf')]
labels = ['1-2 Dormitorios', '3-4 Dormitorios', '5-6 Dormitorios', '7 o más Dormitorios']
df['bedroom_category'] = pd.cut(df['bedrooms'], bins=bins, labels=labels, right=False)


In [198]:
# detallando la variable bedrooms
unique_bedrooms = df['bedroom_category'].unique()
print(unique_bedrooms)
bedrooms_counts = df['bedroom_category'].value_counts()
print(bedrooms_counts)

['3-4 Dormitorios', '5-6 Dormitorios', '1-2 Dormitorios', '7 o más Dormitorios']
Categories (4, object): ['1-2 Dormitorios' < '3-4 Dormitorios' < '5-6 Dormitorios' < '7 o más Dormitorios']
bedroom_category
3-4 Dormitorios        12584
5-6 Dormitorios         8483
7 o más Dormitorios      334
1-2 Dormitorios          212
Name: count, dtype: int64


In [200]:
# voy a escalar algunas variables numericas
# Crear el objeto del escalador
scaler = StandardScaler()
# Definir las características numéricas que deseas escalar
to_scaler = ['sqft_living', 'sqft_above','sqft_basement','house_age','bathrooms']
# Aplicar el escalado a las características numéricas
df[to_scaler] = scaler.fit_transform(df[to_scaler])

In [201]:

# Crear el objeto del codificador one-hot
encoder = OneHotEncoder(drop='first', sparse=False)

# Definir la característica categórica que deseas codificar
categorical_feature = ['view','waterfront','floors','condition','grade_category','bedroom_category']

# Aplicar la codificación one-hot a la característica categórica
encoded_features = encoder.fit_transform(df[categorical_feature])

# Obtener los nombres de las características codificadas
feature_names = encoder.get_feature_names_out(input_features=categorical_feature)

# Crear un DataFrame con las columnas codificadas
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

# Agregar las nuevas columnas al DataFrame original y eliminar la columna original categórica
df = pd.concat([df, encoded_df], axis=1)
df.drop(categorical_feature, axis=1, inplace=True)
print(feature_names)

['view_1' 'view_2' 'view_3' 'view_4' 'waterfront_1' 'floors_1.5'
 'floors_2.0' 'floors_2.5' 'floors_3.0' 'floors_3.5' 'condition_2'
 'condition_3' 'condition_4' 'condition_5'
 'grade_category_Construcción y Diseño Insuficiente' 'grade_category_Otro'
 'grade_category_Promedio' 'bedroom_category_3-4 Dormitorios'
 'bedroom_category_5-6 Dormitorios' 'bedroom_category_7 o más Dormitorios']


c:\Users\David\.virtualenvs\kc_house_data-eeEn6LAL\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### se determino que la variale date almacena la venta de la case, mientras que las columnas yr_built fue el año de constrccion, entonces podemos decir que su resta es la cantidad de años que tenia la casa cuando fue vendida

In [202]:
# seleccionamos las columnas que deseamos usar
selected = ['sqft_living', 'sqft_above','sqft_basement','house_age','bathrooms',
'bedrooms','grade']
features = np.concatenate((selected, feature_names))

In [203]:
# dividimos los datos
y = df["price"]
X = df[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [204]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_test)
    val_mae = mean_absolute_error(val_predictions, y_test)
    print(" MAE para {}: {:,.0f}".format(model_name, val_mae))

In [205]:
model_dtr = DecisionTreeRegressor(random_state=1)
model_dtrmax = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)
model_rf = RandomForestRegressor(random_state=1)
model_knn = KNeighborsRegressor(n_neighbors=5)

In [206]:
train_and_evaluate_model(model_dtr, X_train, y_train, X_test, y_test, "Árbol de Decisión")
train_and_evaluate_model(model_dtrmax, X_train, y_train, X_test, y_test, "Árbol de Decisión (max_leaf_nodes=100)")
train_and_evaluate_model(model_rf, X_train, y_train, X_test, y_test, "Random Forest")
train_and_evaluate_model(model_knn, X_train, y_train, X_test, y_test, "KNN")

 MAE para Árbol de Decisión: 176,608
 MAE para Árbol de Decisión (max_leaf_nodes=100): 137,168
 MAE para Random Forest: 130,224
 MAE para KNN: 137,557
